In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from astropy.table import Table

d = Table.read('/content/drive/MyDrive/catalogs.hdf5')

In [ ]:
import numpy as np

x = np.array(d['image'][:16000]).reshape((-1, 101, 101, 1))
y = np.array(d['is_lens'][:16000]).reshape((-1, 1))

x_test = np.array(d['image'][16000:]).reshape((-1, 101, 101, 1))
y_test = np.array(d['is_lens'][16000:]).reshape((-1, 1))

In [ ]:
vmin = -1e-9
vmax = 1e-9
scale = 100

mask = np.where(x == 100)
mask_test = np.where(x_test == 100)

x[mask] = 0
x_test[mask_test] = 0

x = np.clip(x, vmin, vmax)/vmax * scale
x_test = np.clip(x_test, vmin, vmax)/vmax * scale

x[mask] = 0
x_test[mask_test] = 0

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def resnet_block(input_layer, num_cnn=3, channel=64, block_num=1):
  x = input_layer
  for cnn_num in range(num_cnn):
    identity = x
    x = layers.Conv2D(filters=channel, kernel_size=(3,3), activation='elu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=channel, kernel_size=(3,3), activation='elu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(pool_size=(2,2))(x)

  return x

def build_resnet(input_shape=(101,101,1), num_cnn_list=[3,4,6,3], channel_list=[64,128,256,512], num_classes=2):
  assert len(num_cnn_list) == len(channel_list)

  input_layer = layers.Input(shape=input_shape)
  output = input_layer

  output = layers.Conv2D(filters=64, kernel_size=(3,3), padding='valid')(output)
  output = layers.BatchNormalization()(output)

  output = layers.MaxPooling2D(pool_size=(2,2))(output)

  for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
    output = resnet_block(output, num_cnn=num_cnn, channel=channel, block_num=i)
  output = layers.AveragePooling2D(padding='same')(output)
  output = layers.Flatten()(output)
  output = layers.Dense(512, activation='elu')(output)
  output = layers.Dropout(0.3)(output)
  output = layers.Dense(1, activation='sigmoid')(output)
  
  model = tf.keras.Model(inputs = input_layer, outputs=output)

  return model

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_addons as tfa

resnet = build_resnet()

wd = lambda: 1e-3
optimizer = tfa.optimizers.AdamW(weight_decay=wd)

resnet.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
resnet.fit(x, y, epochs=5)

Epoch 1/20
500/500 [==============================] - 65s 129ms/step - loss: 0.5015 - accuracy: 0.7517
Epoch 2/20
500/500 [==============================] - 65s 130ms/step - loss: 0.4543 - accuracy: 0.7787
Epoch 3/20
500/500 [==============================] - 65s 130ms/step - loss: 0.4313 - accuracy: 0.7933
Epoch 4/20
500/500 [==============================] - 65s 130ms/step - loss: 0.4119 - accuracy: 0.7999
Epoch 5/20
500/500 [==============================] - 65s 131ms/step - loss: 0.4137 - accuracy: 0.7997
Epoch 6/20
500/500 [==============================] - 65s 131ms/step - loss: 0.4099 - accuracy: 0.7987
Epoch 7/20
500/500 [==============================] - 65s 131ms/step - loss: 0.4148 - accuracy: 0.7997
Epoch 8/20
500/500 [==============================] - 65s 131ms/step - loss: 0.4173 - accuracy: 0.7978
Epoch 9/20
500/500 [==============================] - 65s 131ms/step - loss: 0.4029 - accuracy: 0.8034
Epoch 10/20
500/500 [==============================] - 65s 130ms/step - l

In [ ]:
resnet.evaluate(x_test, y_test)

 98/125 [======================>.......] - ETA: 1s - loss: 0.6134 - accuracy: 0.7009